<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Generators/Upload_Processor_Oto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re

In [ ]:
fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-109276.csv")
fda_df = fda_df.sort_values(by=["Active Ingredient UNII(s)","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
import pandas as pd
import re
all_fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-378064.csv")
all_fda_df["SPL Effective Date (Month/Day/Year)"] = pd.to_datetime(all_fda_df["SPL Effective Date (Month/Day/Year)"])
all_fda_df["lower"] = all_fda_df["Generic/Proper Name(s)"].str.lower()
all_fda_df = all_fda_df.sort_values(by=["lower","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
all_fda_df["Active Ingredient(s)"] = all_fda_df["Active Ingredient(s)"].astype(str).dropna()
all_fda_df["Route(s) of Administration"] = all_fda_df["Route(s) of Administration"].astype(str).dropna()
all_fda_df["Active Ingredient UNII(s)"] = all_fda_df["Active Ingredient UNII(s)"].astype(str).dropna()
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient(s)'].str.contains('and|;', regex=True, flags=re.IGNORECASE)]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("TOPICAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("IRRIGATIONAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INTRADERMAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INHALATION")]
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient UNII(s)'].str.contains(";")]
all_fda_df = all_fda_df[all_fda_df["Marketing Category"].isin(["NDA","ANDA","BLA"])]
all_fda_df = all_fda_df.groupby(["lower","Marketing Category"]).first().reset_index()
all_fda_df = all_fda_df.sort_values(["lower","Marketing Category"],ascending=[True,False])
our_df = all_fda_df.groupby("lower").first().reset_index()

<ipython-input-6-aae757967de3>:5: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  all_fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-378064.csv")


In [ ]:
fda_df["Direct FDALabel Link"] = fda_df["FDALabel Link"]
fda_df["lower"] = fda_df["Generic/Proper Name(s)"].str.lower()
fda_df = fda_df[["lower","Direct FDALabel Link"]]
our_df = our_df.merge(fda_df,on="lower",how="left").groupby("lower").first().reset_index()
our_df.loc[our_df["Direct FDALabel Link"].isna(),"Direct FDALabel Link"] = our_df.loc[our_df["Direct FDALabel Link"].isna(),"FDALabel Link"]

In [ ]:
our_df

,lower,Marketing Category,Labeling Type,Dosage Form(s),Route(s) of Administration,Application Number(s),Trade Name,Generic/Proper Name(s),SPL Effective Date (Month/Day/Year),Established Pharmacologic Class(es),...,Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed SPL Link,DailyMed PDF Link,SET ID,Query Link for Entire Search Output,Result Link for Entire Search Output,Export Date (YYYY/MM/DD) and Time for Entire Search Output,Direct FDALabel Link
0,(chloroprocaine hci,NDA,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INFILTRATION; PERINEURAL,009435,(CHLOROPROCAINE HCI,(CHLOROPROCAINE HCI,2024-01-29,Ester Local Anesthetic,...,CHLOROPROCAINE; EDETIC ACID,5YVB0POT2H; 9G34HU7RV0,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,94d13f48-fbca-67b4-e053-2995a90a791b,None,None,None,https://nctr-crs.fda.gov/fdalabel/services/spl...
1,"(coagulation factor ix (recombinant), glycopeg...",BLA,HUMAN PRESCRIPTION DRUG LABEL,KIT,INTRAVENOUS,125611,REBINYN,"(COAGULATION FACTOR IX (RECOMBINANT), GLYCOPEG...",2022-08-11,Human Blood Coagulation Factor,...,COAGULATION FACTOR IX RECOMBINANT HUMAN,382L14738L,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,0ea37235-35fd-410d-b8c4-40ba15fe1294,None,None,None,https://nctr-crs.fda.gov/fdalabel/services/spl...
2,(sotradecol) sodium tetradecyl sulfate,ANDA,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INTRAVENOUS,040541,(SOTRADECOL) SODIUM TETRADECYL SULFATE,(SOTRADECOL) SODIUM TETRADECYL SULFATE,2021-06-25,None,...,TETRADECYL HYDROGEN SULFATE (ESTER),6326W0DRHY,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,c59db022-bda8-1732-e053-2a95a90a24f8,None,None,None,https://nctr-crs.fda.gov/fdalabel/services/spl...
3,0.12% chlorhexidine gluconate,ANDA,HUMAN PRESCRIPTION DRUG LABEL,RINSE,ORAL,077789,Major Oral Rinse,0.12% CHLORHEXIDINE GLUCONATE,2024-01-11,None,...,CHLORHEXIDINE,R4KO0DY52L,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,baad6ebb-ba44-4abe-8882-f2391d17d816,None,None,None,https://nctr-crs.fda.gov/fdalabel/services/spl...
4,0.25% bupivacaine hcl; bupivacaine hydrochloride,ANDA,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INFILTRATION; PERINEURAL,070590;070597,0.25% BUPIVACAINE HCl; BUPIVACAINE HYDROCHLORIDE,0.25% BUPIVACAINE HCL; BUPIVACAINE HYDROCHLORIDE,2024-02-19,Amide Local Anesthetic,...,BUPIVACAINE,Y8335394RO,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,950d82b6-76a1-0c08-e053-2a95a90a3a92,None,None,None,https://nctr-crs.fda.gov/fdalabel/services/spl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,zolmitriptan,NDA,HUMAN PRESCRIPTION DRUG LABEL,"SPRAY, METERED",NASAL,021450,ZOMIG,ZOLMITRIPTAN,2019-05-31,Serotonin-1b and Serotonin-1d Receptor Agonist,...,ZOLMITRIPTAN,2FS66TH3YW,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,84b51cb9-83f3-4a49-7fa3-1adc0f963658,None,None,None,https://fdalabel.fda.gov:8443/fdalabel/service...
2415,zolpidem tartrate,NDA,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,215721,zolpidem tartrate,ZOLPIDEM TARTRATE,2023-05-10,gamma-Aminobutyric Acid-ergic Agonist,...,ZOLPIDEM,7K383OQI23,https://nctr-crs.fda.gov/fdalabel/services/spl...,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,https://dailymed.nlm.nih.gov/dailymed/download...,2f1a3600-9bd6-3651-3ab5-1e4e0b0a3916,None,None,None,https://fdalabel.fda.gov:8443/fdalabel/service...
2416,zolpidem tartrate sublingual,ANDA,HUMAN PRESCRIPTI

In [ ]:
drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
moiety_list = []
fda_label_list = []

for index, row in tqdm(our_df.iterrows(), total=our_df.shape[0]):
    url = row["Direct FDALabel Link"]
    print(url)
    url=url.replace("https://fdalabel.fda.gov:8443","https://nctr-crs.fda.gov")
    if "nctr-crs.fda.gov" in url:
        response = requests.get(url)
        if response.status_code != 200:
          print("Error!")
          continue
        response.raise_for_status()  # Raises an HTTPError for bad responses
        html_content = response.text

    # Parse HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

    # Extract plain text, removing all tags
        plain_text = soup.get_text(separator='\n', strip=True)
    elif "accessdata" in url:
        r = requests.get(url)
        f = io.BytesIO(r.content)

        reader = PyPDF2.PdfReader(f)
        plain_text = "\n".join([reader.pages[i].extract_text().strip() for i in range(0,len(reader.pages))])
        plain_text=plain_text.strip()

    initial_prompt = """Provide a summary of all the parts of the drug label that discuss ototoxicity and auditory risks and ototoxic reactions for this drug.
    In your summary of each sentence, clearly state whether the drug itself was associated with or caused the ototoxicity risk."""


    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Ototoxicity, Less Ototoxicity, or Most Ototoxicity?'
        Now, answer with just one word: No, Less or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Ototoxicity?'
        Now, answer with just one word: Yes or No"""
    initial_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Read through the following drug label"},
        {"role": "user", "content":plain_text},
        {"role": "user", "content":initial_prompt}
        ],
        stream=False, temperature=0,max_tokens=2000
        )
    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["Generic/Proper Name(s)"])
    moiety_list.append(row["Active Moiety UNII(s)"])
    drugs_initial_responses.append(initial_response.choices[0].message.content.strip())
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    initial_prompts.append(initial_prompt)
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    fda_label_list.append(url)

    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Thursday_Oto_direct_4o_ototoxicity_temp_comprehensive_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Thursday_Oto_direct_4o_ototoxicity_comprehensive.csv")

  0%|          | 0/2419 [00:00<?, ?it/s]

https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/94d13f48-fbca-67b4-e053-2995a90a791b/spl-doc


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
  0%|          | 1/2419 [00:09<6:10:02,  9.18s/it]

Less   No
https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/0ea37235-35fd-410d-b8c4-40ba15fe1294/spl-doc


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
  0%|          | 2/2419 [00:14<4:45:44,  7.09s/it]

...

No   No
https://fdalabel.fda.gov:8443/fdalabel/services/spl/set-ids/d12de43e-3ac3-4335-bc85-70d7366a91eb/spl-doc


100%|█████████▉| 2418/2419 [5:08:04<00:08,  8.19s/it]

No   No
https://fdalabel.fda.gov:8443/fdalabel/services/spl/set-ids/f18e53b0-d0bb-422d-8de7-ab64b7292b29/spl-doc


100%|██████████| 2419/2419 [5:08:16<00:00,  7.65s/it]

No   No
